In [2]:
!pip install pandas numpy matplotlib seaborn scikit-learn

In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)

In [74]:
df = pd.read_csv('/content/raw_total_fight_data.csv', sep = ';')

In [75]:
df.head()

,R_fighter,B_fighter,R_KD,B_KD,R_SIG_STR.,B_SIG_STR.,R_SIG_STR_pct,B_SIG_STR_pct,R_TOTAL_STR.,B_TOTAL_STR.,R_TD,B_TD,R_TD_pct,B_TD_pct,R_SUB_ATT,B_SUB_ATT,R_REV,B_REV,R_CTRL,B_CTRL,R_HEAD,B_HEAD,R_BODY,B_BODY,R_LEG,B_LEG,R_DISTANCE,B_DISTANCE,R_CLINCH,B_CLINCH,R_GROUND,B_GROUND,win_by,last_round,last_round_time,Format,Referee,date,location,Fight_type,Winner
0,Cory Sandhagen,Umar Nurmagomedov,0,0,71 of 159,98 of 187,44%,52%,80 of 172,123 of 219,0 of 2,5 of 13,0%,38%,0,0,0,0,0:12,5:11,26 of 94,48 of 124,18 of 32,27 of 39,27 of 33,23 of 24,71 of 158,96 of 182,0 of 1,2 of 5,0 of 0,0 of 0,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Marc Goddard,"August 03, 2024","Abu Dhabi, Abu Dhabi, United Arab Emirates",Bantamweight Bout,Umar Nurmagomedov
1,Azamat Murzakanov,Alonzo Menifield,1,0,45 of 85,15 of 74,52%,20%,48 of 88,19 of 80,0 of 1,0 of 0,0%,---,0,0,0,0,0:44,0:51,34 of 69,14 of 73,11 of 16,1 of 1,0 of 0,0 of 0,43 of 82,15 of 73,0 of 1,0 of 1,2 of 2,0 of 0,KO/TKO,2,3:18,3 Rnd (5-5-5),Daniel Movahedi,"August 03, 2024","Abu Dhabi, Abu Dhabi, United Arab Emirates",Light Heavyweight Bout,Azamat Murzakanov
2,Tony Ferguson,Michael Chiesa,0,0,10 of 18,4 of 25,55%,16%,10 of 18,4 of 25,0 of 0,1 of 1,---,100%,0,1,0,0,0:00,0:34,4 of 9,3 of 23,1 of 3,1 of 2,5 of 6,0 of 0,10 of 18,4 of 25,0 of 0,0 of 0,0 of 0,0 of 0,Submission,1,3:44,3 Rnd (5-5-5),Marc Goddard,"August 03, 2024","Abu Dhabi, Abu Dhabi, United Arab Emirates",Welterweight Bout,Michael Chiesa
3,Shamil Gaziev,Don'Tale Mayes,0,0,31 of 46,27 of 58,67%,46%,89 of 114,57 of 95,2 of 3,0 of 2,66%,0%,0,0,0,0,11:58,0:00,22 of 37,18 of 45,8 of 8,5 of 9,1 of 1,4 of 4,12 of 25,25 of 53,15 of 17,2 of 5,4 of 4,0 of 0,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Keith Peterson,"August 03, 2024","Abu Dhabi, Abu Dhabi, United Arab Emirates",Heavyweight Bout,Shamil Gaziev
4,Joel Alvarez,Elves Brener,1,0,95 of 141,45 of 97,67%,46%,114 of 164,45 of 98,0 of 1,0 of 5,0%,0%,1,0,0,0,0:51,0:22,67 of 111,17 of 58,24 of 26,7 of 16,4 of 4,21 of 23,71 of 112,45 of 96,24 of 29,0 of 1,0 of 0,0 of 0,KO/TKO,3,3:36,3 Rnd (5-5-5),Lukasz Bosacki,"August 03, 2024","Abu Dhabi, Abu Dhabi, United Arab Emirates",Lightweight Bout,Joel Alvarez


In [76]:
df.shape

(7713, 41)

#Data Cleaning

In [77]:
columns_to_remove_1=['R_REV', 'B_REV', 'R_HEAD', 'B_HEAD', 'R_BODY', 'B_BODY', 'R_LEG', 'B_LEG',
                       'R_DISTANCE', 'B_DISTANCE', 'R_CLINCH', 'B_CLINCH']
df.drop(columns_to_remove_1, axis=1,inplace=True)

columns_to_remove_2=['Referee', 'location']
df.drop(columns_to_remove_2, axis=1,inplace=True)

df['Format']=df['Format'].str.extract(r'(\d+)')
df.rename(columns={'Format': 'No_of_rounds'},inplace=True)

df.rename(columns={'R_KD': 'R_Knockdown'},inplace=True)
df.rename(columns={'B_KD': 'B_Knockdown'},inplace=True)
df.rename(columns={'R_SIG_STR.': 'R_Significant_Strikes'},inplace=True)
df.rename(columns={'B_SIG_STR.': 'B_Significant_Strikes'},inplace=True)
df.rename(columns={'R_SIG_STR_pct': 'R_Significant_Strike_Percent'},inplace=True)
df.rename(columns={'B_SIG_STR_pct': 'B_Significant_Strike_Percent'},inplace=True)
df.rename(columns={'R_TOTAL_STR.': 'R_Total_Strikes'},inplace=True)
df.rename(columns={'B_TOTAL_STR.': 'B_Total_Strikes'},inplace=True)
df.rename(columns={'R_TD': 'R_Takedowns'},inplace=True)
df.rename(columns={'B_TD': 'B_Takedowns'},inplace=True)
df.rename(columns={'R_TD_pct': 'R_Takedown_Percent'},inplace=True)
df.rename(columns={'B_TD_pct': 'B_Takedown_Percent'},inplace=True)
df.rename(columns={'R_SUB_ATT': 'R_Submission_Attempt'},inplace=True)
df.rename(columns={'B_SUB_ATT': 'B_Submission_Attempt'},inplace=True)
df.rename(columns={'R_CTRL': 'R_Ground_Control'},inplace=True)
df.rename(columns={'B_CTRL': 'B_Ground_Control'},inplace=True)
df.rename(columns={'R_GROUND': 'R_Ground_Strikes'},inplace=True)
df.rename(columns={'B_GROUND': 'B_Ground_Strikes'},inplace=True)

df.head()

,R_fighter,B_fighter,R_Knockdown,B_Knockdown,R_Significant_Strikes,B_Significant_Strikes,R_Significant_Strike_Percent,B_Significant_Strike_Percent,R_Total_Strikes,B_Total_Strikes,R_Takedowns,B_Takedowns,R_Takedown_Percent,B_Takedown_Percent,R_Submission_Attempt,B_Submission_Attempt,R_Ground_Control,B_Ground_Control,R_Ground_Strikes,B_Ground_Strikes,win_by,last_round,last_round_time,No_of_rounds,date,Fight_type,Winner
0,Cory Sandhagen,Umar Nurmagomedov,0,0,71 of 159,98 of 187,44%,52%,80 of 172,123 of 219,0 of 2,5 of 13,0%,38%,0,0,0:12,5:11,0 of 0,0 of 0,Decision - Unanimous,5,5:00,5,"August 03, 2024",Bantamweight Bout,Umar Nurmagomedov
1,Azamat Murzakanov,Alonzo Menifield,1,0,45 of 85,15 of 74,52%,20%,48 of 88,19 of 80,0 of 1,0 of 0,0%,---,0,0,0:44,0:51,2 of 2,0 of 0,KO/TKO,2,3:18,3,"August 03, 2024",Light Heavyweight Bout,Azamat Murzakanov
2,Tony Ferguson,Michael Chiesa,0,0,10 of 18,4 of 25,55%,16%,10 of 18,4 of 25,0 of 0,1 of 1,---,100%,0,1,0:00,0:34,0 of 0,0 of 0,Submission,1,3:44,3,"August 03, 2024",Welterweight Bout,Michael Chiesa
3,Shamil Gaziev,Don'Tale Mayes,0,0,31 of 46,27 of 58,67%,46%,89 of 114,57 of 95,2 of 3,0 of 2,66%,0%,0,0,11:58,0:00,4 of 4,0 of 0,Decision - Unanimous,3,5:00,3,"August 03, 2024",Heavyweight Bout,Shamil Gaziev
4,Joel Alvarez,Elves Brener,1,0,95 of 141,45 of 97,67%,46%,114 of 164,45 of 98,0 of 1,0 of 5,0%,0%,1,0,0:51,0:22,0 of 0,0 of 0,KO/TKO,3,3:36,3,"August 03, 2024",Lightweight Bout,Joel Alvarez


In [78]:
df[['R_Significant_Strikes_Landed','R_Significant_Strikes_Attempted']]=df['R_Significant_Strikes'].str.split(' of ',expand=True)
df[['B_Significant_Strikes_Landed','B_Significant_Strikes_Attempted']]=df['B_Significant_Strikes'].str.split(' of ',expand=True)

columns_to_remove_3=['R_Significant_Strikes', 'B_Significant_Strikes']

df.drop(columns_to_remove_3,axis=1,inplace=True)

df.head()

df[['R_Total_Strikes_Landed','R_Total_Strikes_Attempted']]=df['R_Total_Strikes'].str.split(' of ',expand=True)
df[['B_Total_Strikes_Landed','B_Total_Strikes_Attempted']]=df['B_Total_Strikes'].str.split(' of ',expand=True)

columns_to_remove_4=['R_Total_Strikes','B_Total_Strikes']

df.drop(columns_to_remove_4, axis=1,inplace=True)

df[['R_Takedowns_Landed','R_Takedowns_Attempted']]=df['R_Takedowns'].str.split(' of ',expand=True)
df[['B_Takedowns_Landed','B_Takedowns_Attempted']]=df['B_Takedowns'].str.split(' of ',expand=True)

columns_to_remove_5=['R_Takedowns','B_Takedowns']

df.drop(columns_to_remove_5,axis=1,inplace=True)

df[['R_Ground_Strikes_Landed','R_Ground_Strikes_Attempted']]=df['R_Ground_Strikes'].str.split(' of ',expand=True)
df[['B_Ground_Strikes_Landed','B_Ground_Strikes_Attempted']]=df['B_Ground_Strikes'].str.split(' of ',expand=True)

columns_to_remove_6=['R_Ground_Strikes','B_Ground_Strikes']

df.drop(columns_to_remove_6,axis=1,inplace=True)

df['R_Significant_Strike_Percent']=df['R_Significant_Strike_Percent'].str.replace('%', '')
df['B_Significant_Strike_Percent']=df['B_Significant_Strike_Percent'].str.replace('%', '')
df['R_Takedown_Percent']=df['R_Takedown_Percent'].str.replace('%', '')
df['B_Takedown_Percent']=df['B_Takedown_Percent'].str.replace('%', '')

df['R_Significant_Strike_Percent']=df['R_Significant_Strike_Percent'].str.replace('---', '0')
df['B_Significant_Strike_Percent']=df['B_Significant_Strike_Percent'].str.replace('---', '0')
df['R_Takedown_Percent']=df['R_Takedown_Percent'].str.replace('---', '0')
df['B_Takedown_Percent']=df['B_Takedown_Percent'].str.replace('---', '0')

df.shape

(7713, 35)

In [79]:
df.head()

,R_fighter,B_fighter,R_Knockdown,B_Knockdown,R_Significant_Strike_Percent,B_Significant_Strike_Percent,R_Takedown_Percent,B_Takedown_Percent,R_Submission_Attempt,B_Submission_Attempt,R_Ground_Control,B_Ground_Control,win_by,last_round,last_round_time,No_of_rounds,date,Fight_type,Winner,R_Significant_Strikes_Landed,R_Significant_Strikes_Attempted,B_Significant_Strikes_Landed,B_Significant_Strikes_Attempted,R_Total_Strikes_Landed,R_Total_Strikes_Attempted,B_Total_Strikes_Landed,B_Total_Strikes_Attempted,R_Takedowns_Landed,R_Takedowns_Attempted,B_Takedowns_Landed,B_Takedowns_Attempted,R_Ground_Strikes_Landed,R_Ground_Strikes_Attempted,B_Ground_Strikes_Landed,B_Ground_Strikes_Attempted
0,Cory Sandhagen,Umar Nurmagomedov,0,0,44,52,0,38,0,0,0:12,5:11,Decision - Unanimous,5,5:00,5,"August 03, 2024",Bantamweight Bout,Umar Nurmagomedov,71,159,98,187,80,172,123,219,0,2,5,13,0,0,0,0
1,Azamat Murzakanov,Alonzo Menifield,1,0,52,20,0,0,0,0,0:44,0:51,KO/TKO,2,3:18,3,"August 03, 2024",Light Heavyweight Bout,Azamat Murzakanov,45,85,15,74,48,88,19,80,0,1,0,0,2,2,0,0
2,Tony Ferguson,Michael Chiesa,0,0,55,16,0,100,0,1,0:00,0:34,Submission,1,3:44,3,"August 03, 2024",Welterweight Bout,Michael Chiesa,10,18,4,25,10,18,4,25,0,0,1,1,0,0,0,0
3,Shamil Gaziev,Don'Tale Mayes,0,0,67,46,66,0,0,0,11:58,0:00,Decision - Unanimous,3,5:00,3,"August 03, 2024",Heavyweight Bout,Shamil Gaziev,31,46,27,58,89,114,57,95,2,3,0,2,4,4,0,0
4,Joel Alvarez,Elves Brener,1,0,67,46,0,0,1,0,0:51,0:22,KO/TKO,3,3:36,3,"August 03, 2024",Lightweight Bout,Joel Alvarez,95,141,45,97,114,164,45,98,0,1,0,5,0,0,0,0


In [80]:
print(df['date'].dtype)

object


In [81]:
df['date']=pd.to_datetime(df['date'],format='%B %d, %Y')
df['date']=df['date'].dt.strftime('%Y-%m-%d')

date_limit='2001-04-01'
df=df.loc[df['date']>date_limit]

In [82]:
df.shape

(7459, 35)

In [83]:
df.head()

,R_fighter,B_fighter,R_Knockdown,B_Knockdown,R_Significant_Strike_Percent,B_Significant_Strike_Percent,R_Takedown_Percent,B_Takedown_Percent,R_Submission_Attempt,B_Submission_Attempt,R_Ground_Control,B_Ground_Control,win_by,last_round,last_round_time,No_of_rounds,date,Fight_type,Winner,R_Significant_Strikes_Landed,R_Significant_Strikes_Attempted,B_Significant_Strikes_Landed,B_Significant_Strikes_Attempted,R_Total_Strikes_Landed,R_Total_Strikes_Attempted,B_Total_Strikes_Landed,B_Total_Strikes_Attempted,R_Takedowns_Landed,R_Takedowns_Attempted,B_Takedowns_Landed,B_Takedowns_Attempted,R_Ground_Strikes_Landed,R_Ground_Strikes_Attempted,B_Ground_Strikes_Landed,B_Ground_Strikes_Attempted
0,Cory Sandhagen,Umar Nurmagomedov,0,0,44,52,0,38,0,0,0:12,5:11,Decision - Unanimous,5,5:00,5,2024-08-03,Bantamweight Bout,Umar Nurmagomedov,71,159,98,187,80,172,123,219,0,2,5,13,0,0,0,0
1,Azamat Murzakanov,Alonzo Menifield,1,0,52,20,0,0,0,0,0:44,0:51,KO/TKO,2,3:18,3,2024-08-03,Light Heavyweight Bout,Azamat Murzakanov,45,85,15,74,48,88,19,80,0,1,0,0,2,2,0,0
2,Tony Ferguson,Michael Chiesa,0,0,55,16,0,100,0,1,0:00,0:34,Submission,1,3:44,3,2024-08-03,Welterweight Bout,Michael Chiesa,10,18,4,25,10,18,4,25,0,0,1,1,0,0,0,0
3,Shamil Gaziev,Don'Tale Mayes,0,0,67,46,66,0,0,0,11:58,0:00,Decision - Unanimous,3,5:00,3,2024-08-03,Heavyweight Bout,Shamil Gaziev,31,46,27,58,89,114,57,95,2,3,0,2,4,4,0,0
4,Joel Alvarez,Elves Brener,1,0,67,46,0,0,1,0,0:51,0:22,KO/TKO,3,3:36,3,2024-08-03,Lightweight Bout,Joel Alvarez,95,141,45,97,114,164,45,98,0,1,0,5,0,0,0,0


In [84]:
df.win_by.unique()

array(['Decision - Unanimous', 'KO/TKO', 'Submission', 'Decision - Split',
       "TKO - Doctor's Stoppage", 'Could Not Continue',
       'Decision - Majority', 'DQ', 'Overturned'], dtype=object)

In [85]:
df=df[df['Winner'].notna()]

print("Number of total NaN values:",df.isna().sum().sum())
print("Number of NaN in each column: ")

nan_count_list=list()

for column_index, column_name in enumerate(df.columns):
  nan_count=df[column_name].isna().sum()
  nan_count_list.append((column_index,nan_count))

sorted_nan_count=sorted(nan_count_list)

for _ in range(df.columns.size):
  column_index=sorted_nan_count[_][0]
  nan_total=sorted_nan_count[_][1]
  print(df.columns[column_index],nan_total)

Number of total NaN values: 0
Number of NaN in each column: 
R_fighter 0
B_fighter 0
R_Knockdown 0
B_Knockdown 0
R_Significant_Strike_Percent 0
B_Significant_Strike_Percent 0
R_Takedown_Percent 0
B_Takedown_Percent 0
R_Submission_Attempt 0
B_Submission_Attempt 0
R_Ground_Control 0
B_Ground_Control 0
win_by 0
last_round 0
last_round_time 0
No_of_rounds 0
date 0
Fight_type 0
Winner 0
R_Significant_Strikes_Landed 0
R_Significant_Strikes_Attempted 0
B_Significant_Strikes_Landed 0
B_Significant_Strikes_Attempted 0
R_Total_Strikes_Landed 0
R_Total_Strikes_Attempted 0
B_Total_Strikes_Landed 0
B_Total_Strikes_Attempted 0
R_Takedowns_Landed 0
R_Takedowns_Attempted 0
B_Takedowns_Landed 0
B_Takedowns_Attempted 0
R_Ground_Strikes_Landed 0
R_Ground_Strikes_Attempted 0
B_Ground_Strikes_Landed 0
B_Ground_Strikes_Attempted 0


#Feature Engineering

In [86]:
print(df.dtypes)

R_fighter                          object
B_fighter                          object
R_Knockdown                         int64
B_Knockdown                         int64
R_Significant_Strike_Percent       object
B_Significant_Strike_Percent       object
R_Takedown_Percent                 object
B_Takedown_Percent                 object
R_Submission_Attempt                int64
B_Submission_Attempt                int64
R_Ground_Control                   object
B_Ground_Control                   object
win_by                             object
last_round                          int64
last_round_time                    object
No_of_rounds                       object
date                               object
Fight_type                         object
Winner                             object
R_Significant_Strikes_Landed       object
R_Significant_Strikes_Attempted    object
B_Significant_Strikes_Landed       object
B_Significant_Strikes_Attempted    object
R_Total_Strikes_Landed            

In [87]:
def calculate_duration(round_time):
  time=round_time.split(':')
  minutes=int(time[0])
  seconds=int(time[1])
  total_seconds=minutes*60+seconds
  return total_seconds

In [88]:
df['last_round_time']=df['last_round_time'].apply(calculate_duration)
df['time_fought']=(df['last_round']-1)*5*60+df['last_round_time']

to_drop=['last_round','last_round_time']
df.drop(to_drop,axis=1,inplace=True)

In [89]:
df.head()

,R_fighter,B_fighter,R_Knockdown,B_Knockdown,R_Significant_Strike_Percent,B_Significant_Strike_Percent,R_Takedown_Percent,B_Takedown_Percent,R_Submission_Attempt,B_Submission_Attempt,R_Ground_Control,B_Ground_Control,win_by,No_of_rounds,date,Fight_type,Winner,R_Significant_Strikes_Landed,R_Significant_Strikes_Attempted,B_Significant_Strikes_Landed,B_Significant_Strikes_Attempted,R_Total_Strikes_Landed,R_Total_Strikes_Attempted,B_Total_Strikes_Landed,B_Total_Strikes_Attempted,R_Takedowns_Landed,R_Takedowns_Attempted,B_Takedowns_Landed,B_Takedowns_Attempted,R_Ground_Strikes_Landed,R_Ground_Strikes_Attempted,B_Ground_Strikes_Landed,B_Ground_Strikes_Attempted,time_fought
0,Cory Sandhagen,Umar Nurmagomedov,0,0,44,52,0,38,0,0,0:12,5:11,Decision - Unanimous,5,2024-08-03,Bantamweight Bout,Umar Nurmagomedov,71,159,98,187,80,172,123,219,0,2,5,13,0,0,0,0,1500
1,Azamat Murzakanov,Alonzo Menifield,1,0,52,20,0,0,0,0,0:44,0:51,KO/TKO,3,2024-08-03,Light Heavyweight Bout,Azamat Murzakanov,45,85,15,74,48,88,19,80,0,1,0,0,2,2,0,0,498
2,Tony Ferguson,Michael Chiesa,0,0,55,16,0,100,0,1,0:00,0:34,Submission,3,2024-08-03,Welterweight Bout,Michael Chiesa,10,18,4,25,10,18,4,25,0,0,1,1,0,0,0,0,224
3,Shamil Gaziev,Don'Tale Mayes,0,0,67,46,66,0,0,0,11:58,0:00,Decision - Unanimous,3,2024-08-03,Heavyweight Bout,Shamil Gaziev,31,46,27,58,89,114,57,95,2,3,0,2,4,4,0,0,900
4,Joel Alvarez,Elves Brener,1,0,67,46,0,0,1,0,0:51,0:22,KO/TKO,3,2024-08-03,Lightweight Bout,Joel Alvarez,95,141,45,97,114,164,45,98,0,1,0,5,0,0,0,0,816


In [90]:
df_DQ=df.loc[df['win_by']=='DQ']
df.drop(df_DQ.index,inplace=True)

In [91]:
R_subset=['R_fighter','R_Knockdown','R_Significant_Strike_Percent','R_Takedown_Percent', 'R_Submission_Attempt','R_Ground_Control', 'win_by', 'No_of_rounds', 'date', 'Fight_type',
            'R_Significant_Strikes_Landed', 'R_Significant_Strikes_Attempted', 'R_Total_Strikes_Landed', 'R_Total_Strikes_Attempted', 'R_Takedowns_Landed', 'R_Takedowns_Attempted',
            'R_Ground_Strikes_Landed', 'time_fought','Winner']
B_subset = ['B_fighter', 'B_Knockdown', 'B_Significant_Strike_Percent', 'B_Takedown_Percent', 'B_Submission_Attempt',
            'B_Ground_Control', 'win_by', 'No_of_rounds', 'date', 'Fight_type', 'B_Significant_Strikes_Landed',
            'B_Significant_Strikes_Attempted', 'B_Total_Strikes_Landed', 'B_Total_Strikes_Attempted',
            'B_Takedowns_Landed', 'B_Takedowns_Attempted', 'B_Ground_Strikes_Landed', 'time_fought', 'Winner']


R_df=df[R_subset]
R_df.columns=R_df.columns.str.replace('R_','')
R_df['Winner'] = np.where(R_df['Winner'] == R_df['fighter'], 1, 0)

B_df=df[B_subset]
B_df.columns=B_df.columns.str.replace('B_','')
B_df['Winner'] = np.where(B_df['Winner'] == B_df['fighter'], 1, 0)

new_df=pd.concat([R_df,B_df],axis=0)
new_df.sort_values(by=['date'],ascending=False,inplace=True)
new_df.reset_index(drop=True,inplace=True)

ufc=new_df.copy()
ufc['Ground_Control']=ufc['Ground_Control'].apply(calculate_duration)

<ipython-input-91-8a6a7bfbe987>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  R_df['Winner'] = np.where(R_df['Winner'] == R_df['fighter'], 1, 0)
<ipython-input-91-8a6a7bfbe987>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  B_df['Winner'] = np.where(B_df['Winner'] == B_df['fighter'], 1, 0)


In [92]:
ufc.head()

,fighter,Knockdown,Significant_Strike_Percent,Takedown_Percent,Submission_Attempt,Ground_Control,win_by,No_of_rounds,date,Fight_type,Significant_Strikes_Landed,Significant_Strikes_Attempted,Total_Strikes_Landed,Total_Strikes_Attempted,Takedowns_Landed,Takedowns_Attempted,Ground_Strikes_Landed,time_fought,Winner
0,Cory Sandhagen,0,44,0,0,12,Decision - Unanimous,5,2024-08-03,Bantamweight Bout,71,159,80,172,0,2,0,1500,0
1,Azamat Murzakanov,1,52,0,0,44,KO/TKO,3,2024-08-03,Light Heavyweight Bout,45,85,48,88,0,1,2,498,1
2,Denis Tiuliulin,0,37,60,0,147,Decision - Unanimous,3,2024-08-03,Middleweight Bout,26,70,183,246,3,5,0,900,0
3,Sam Hughes,0,50,0,0,134,Decision - Split,3,2024-08-03,Women's Strawweight Bout,97,193,139,240,0,0,5,900,1
4,Deiveson Figueiredo,1,50,14,0,150,Decision - Unanimous,3,2024-08-03,Bantamweight Bout,45,90,59,104,2,14,2,900,1


In [93]:
ufc.win_by.unique()

array(['Decision - Unanimous', 'KO/TKO', 'Decision - Split', 'Submission',
       "TKO - Doctor's Stoppage", 'Decision - Majority'], dtype=object)

In [94]:
mapping={
    'KO/TKO':10,
    'Submission':9,
    "TKO - Doctor's Stoppage":9,
    'Decision - Unanimous':7,
    'Decision - Majority':6,
    'Decision - Split':7
}

ufc=ufc.replace(mapping)

In [95]:
ufc.head()

,fighter,Knockdown,Significant_Strike_Percent,Takedown_Percent,Submission_Attempt,Ground_Control,win_by,No_of_rounds,date,Fight_type,Significant_Strikes_Landed,Significant_Strikes_Attempted,Total_Strikes_Landed,Total_Strikes_Attempted,Takedowns_Landed,Takedowns_Attempted,Ground_Strikes_Landed,time_fought,Winner
0,Cory Sandhagen,0,44,0,0,12,7,5,2024-08-03,Bantamweight Bout,71,159,80,172,0,2,0,1500,0
1,Azamat Murzakanov,1,52,0,0,44,10,3,2024-08-03,Light Heavyweight Bout,45,85,48,88,0,1,2,498,1
2,Denis Tiuliulin,0,37,60,0,147,7,3,2024-08-03,Middleweight Bout,26,70,183,246,3,5,0,900,0
3,Sam Hughes,0,50,0,0,134,7,3,2024-08-03,Women's Strawweight Bout,97,193,139,240,0,0,5,900,1
4,Deiveson Figueiredo,1,50,14,0,150,7,3,2024-08-03,Bantamweight Bout,45,90,59,104,2,14,2,900,1


In [96]:
ufc.Fight_type.unique()

array(['Bantamweight Bout', 'Light Heavyweight Bout', 'Middleweight Bout',
       "Women's Strawweight Bout", 'Lightweight Bout', 'Heavyweight Bout',
       'Welterweight Bout', 'Featherweight Bout',
       'UFC Welterweight Title Bout', 'Flyweight Bout',
       'UFC Interim Heavyweight Title Bout', "Women's Flyweight Bout",
       "Women's Bantamweight Bout", 'UFC Light Heavyweight Title Bout',
       'Catch Weight Bout', 'UFC Lightweight Title Bout',
       'UFC Flyweight Title Bout', "UFC Women's Strawweight Title Bout",
       'UFC Bantamweight Title Bout', 'UFC Featherweight Title Bout',
       'UFC Middleweight Title Bout',
       "UFC Women's Bantamweight Title Bout",
       "Women's Featherweight Bout", "UFC Women's Flyweight Title Bout",
       'UFC Heavyweight Title Bout',
       'UFC Interim Featherweight Title Bout',
       'Road To UFC 1 Lightweight Tournament Title Bout',
       'Road To UFC 1 Featherweight Tournament Title Bout',
       'Road To UFC 1 Bantamweight Tourna

In [97]:
ufc['Fight_type']=ufc['Fight_type'].str.replace('Bout', '')
ufc['Fight_type']=ufc['Fight_type'].str.replace('Title', '')
ufc['Fight_type']=ufc['Fight_type'].str.replace('Tournament', '')
ufc['Fight_type']=ufc['Fight_type'].str.replace('Ultimate Fighter', '')
ufc['Fight_type']=ufc['Fight_type'].str.replace('UFC', '')
ufc['Fight_type']=ufc['Fight_type'].str.replace('Interim', '')
ufc['Fight_type']=ufc['Fight_type'].str.replace('Brazil', '')
ufc['Fight_type']=ufc['Fight_type'].str.replace('America', '')
ufc['Fight_type']=ufc['Fight_type'].str.replace('China', '')
ufc['Fight_type']=ufc['Fight_type'].str.replace('TUF', '')
ufc['Fight_type']=ufc['Fight_type'].str.replace('Australia', '')
ufc['Fight_type']=ufc['Fight_type'].str.replace('Nations', '')
ufc['Fight_type']=ufc['Fight_type'].str.replace('Canada', '')
ufc['Fight_type']=ufc['Fight_type'].str.replace('vs.', '')
ufc['Fight_type']=ufc['Fight_type'].str.replace('UK', '')
ufc['Fight_type']=ufc['Fight_type'].str.replace('Latin', '')
ufc['Fight_type']=ufc['Fight_type'].str.replace("Women's", 'W')

ufc['Fight_type']=ufc['Fight_type'].apply(lambda x: re.sub(r'\d+', ' ', x))
ufc['Fight_type']=ufc['Fight_type'].str.strip()

In [98]:
ufc.head()

,fighter,Knockdown,Significant_Strike_Percent,Takedown_Percent,Submission_Attempt,Ground_Control,win_by,No_of_rounds,date,Fight_type,Significant_Strikes_Landed,Significant_Strikes_Attempted,Total_Strikes_Landed,Total_Strikes_Attempted,Takedowns_Landed,Takedowns_Attempted,Ground_Strikes_Landed,time_fought,Winner
0,Cory Sandhagen,0,44,0,0,12,7,5,2024-08-03,Bantamweight,71,159,80,172,0,2,0,1500,0
1,Azamat Murzakanov,1,52,0,0,44,10,3,2024-08-03,Light Heavyweight,45,85,48,88,0,1,2,498,1
2,Denis Tiuliulin,0,37,60,0,147,7,3,2024-08-03,Middleweight,26,70,183,246,3,5,0,900,0
3,Sam Hughes,0,50,0,0,134,7,3,2024-08-03,W Strawweight,97,193,139,240,0,0,5,900,1
4,Deiveson Figueiredo,1,50,14,0,150,7,3,2024-08-03,Bantamweight,45,90,59,104,2,14,2,900,1


In [99]:
data_types=ufc.dtypes
print(data_types)

fighter                          object
Knockdown                         int64
Significant_Strike_Percent       object
Takedown_Percent                 object
Submission_Attempt                int64
Ground_Control                    int64
win_by                            int64
No_of_rounds                     object
date                             object
Fight_type                       object
Significant_Strikes_Landed       object
Significant_Strikes_Attempted    object
Total_Strikes_Landed             object
Total_Strikes_Attempted          object
Takedowns_Landed                 object
Takedowns_Attempted              object
Ground_Strikes_Landed            object
time_fought                       int64
Winner                            int64
dtype: object


In [100]:
convert_columns=['Significant_Strike_Percent', 'Takedown_Percent', 'win_by',
                     'No_of_rounds','Significant_Strikes_Landed','Significant_Strikes_Attempted',
                     'Total_Strikes_Landed','Total_Strikes_Attempted','Takedowns_Landed',
                      'Takedowns_Attempted','Ground_Strikes_Landed']

ufc[convert_columns]=ufc[convert_columns].astype('int64')

In [101]:
ufc

,fighter,Knockdown,Significant_Strike_Percent,Takedown_Percent,Submission_Attempt,Ground_Control,win_by,No_of_rounds,date,Fight_type,Significant_Strikes_Landed,Significant_Strikes_Attempted,Total_Strikes_Landed,Total_Strikes_Attempted,Takedowns_Landed,Takedowns_Attempted,Ground_Strikes_Landed,time_fought,Winner
0,Cory Sandhagen,0,44,0,0,12,7,5,2024-08-03,Bantamweight,71,159,80,172,0,2,0,1500,0
1,Azamat Murzakanov,1,52,0,0,44,10,3,2024-08-03,Light Heavyweight,45,85,48,88,0,1,2,498,1
2,Denis Tiuliulin,0,37,60,0,147,7,3,2024-08-03,Middleweight,26,70,183,246,3,5,0,900,0
3,Sam Hughes,0,50,0,0,134,7,3,2024-08-03,W Strawweight,97,193,139,240,0,0,5,900,1
4,Deiveson Figueiredo,1,50,14,0,150,7,3,2024-08-03,Bantamweight,45,90,59,104,2,14,2,900,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14601,Pat Miletich,0,36,0,0,44,9,5,2001-05-04,Welterweight,28,76,50,100,0,0,1,770,0
14602,Pedro Rizzo,1,38,0,0,256,7,5,2001-05-04,Heavyweight,42,108,76,144,0,0,6,1500,0
14603,Pete Williams,0,66,25,0,287,10,3,2001-05-04,Heavyweight,6,9,14,18,1,4,5,388,0
14604,Steve Berger,0,11,0,3,67,7,3,2001-05-04,Welterweight,2,18,108,129,0,0,0,900,0


In [102]:
ufc.isna().sum()

,0
fighter,0
Knockdown,0
Significant_Strike_Percent,0
Takedown_Percent,0
Submission_Attempt,0
Ground_Control,0
win_by,0
No_of_rounds,0
date,0
Fight_type,0


In [103]:
def assign_gender(row):
  women_weight_classes=['W Strawweight', 'W Flyweight', 'W Bantamweight', 'W Featherweight']
  if row['Fight_type'] in women_weight_classes:
    return 0
  else:
    return 1

ufc['Gender']=ufc.apply(assign_gender,axis=1)
column_titles=list(ufc.columns[:-2])+['Gender','Winner']
ufc=ufc.reindex(columns=column_titles)

In [104]:
ufc

,fighter,Knockdown,Significant_Strike_Percent,Takedown_Percent,Submission_Attempt,Ground_Control,win_by,No_of_rounds,date,Fight_type,Significant_Strikes_Landed,Significant_Strikes_Attempted,Total_Strikes_Landed,Total_Strikes_Attempted,Takedowns_Landed,Takedowns_Attempted,Ground_Strikes_Landed,time_fought,Gender,Winner
0,Cory Sandhagen,0,44,0,0,12,7,5,2024-08-03,Bantamweight,71,159,80,172,0,2,0,1500,1,0
1,Azamat Murzakanov,1,52,0,0,44,10,3,2024-08-03,Light Heavyweight,45,85,48,88,0,1,2,498,1,1
2,Denis Tiuliulin,0,37,60,0,147,7,3,2024-08-03,Middleweight,26,70,183,246,3,5,0,900,1,0
3,Sam Hughes,0,50,0,0,134,7,3,2024-08-03,W Strawweight,97,193,139,240,0,0,5,900,0,1
4,Deiveson Figueiredo,1,50,14,0,150,7,3,2024-08-03,Bantamweight,45,90,59,104,2,14,2,900,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14601,Pat Miletich,0,36,0,0,44,9,5,2001-05-04,Welterweight,28,76,50,100,0,0,1,770,1,0
14602,Pedro Rizzo,1,38,0,0,256,7,5,2001-05-04,Heavyweight,42,108,76,144,0,0,6,1500,1,0
14603,Pete Williams,0,66,25,0,287,10,3,2001-05-04,Heavyweight,6,9,14,18,1,4,5,388,1,0
14604,Steve Berger,0,11,0,3,67,7,3,2001-05-04,Welterweight,2,18,108,129,0,0,0,900,1,0


In [105]:
ufc.drop(ufc[ufc['Fight_type'] == 'Catch Weight'].index, inplace=True)
ufc.drop(ufc[ufc['Fight_type'].str.startswith('Road To')].index, inplace=True)
ufc.drop(ufc[ufc['Fight_type'].str.startswith('Road to')].index, inplace=True)

In [106]:
weight_classes={
    'W Strawweight':115,
    'W Flyweight':125,
    'W Bantamweight':135,
    'W Featherweight':145,
    'Flyweight':125,
    'Bantamweight':135,
    'Featherweight':145,
    'Lightweight':155,
    'Welterweight':170,
    'Middleweight':185,
    'Light Heavyweight':205,
    'Heavyweight':265,
}

ufc=ufc.replace(weight_classes)
ufc.rename(columns={'Fight_type': 'Weight Division'},inplace=True)

In [107]:
ufc.head()

,fighter,Knockdown,Significant_Strike_Percent,Takedown_Percent,Submission_Attempt,Ground_Control,win_by,No_of_rounds,date,Weight Division,Significant_Strikes_Landed,Significant_Strikes_Attempted,Total_Strikes_Landed,Total_Strikes_Attempted,Takedowns_Landed,Takedowns_Attempted,Ground_Strikes_Landed,time_fought,Gender,Winner
0,Cory Sandhagen,0,44,0,0,12,7,5,2024-08-03,135,71,159,80,172,0,2,0,1500,1,0
1,Azamat Murzakanov,1,52,0,0,44,10,3,2024-08-03,205,45,85,48,88,0,1,2,498,1,1
2,Denis Tiuliulin,0,37,60,0,147,7,3,2024-08-03,185,26,70,183,246,3,5,0,900,1,0
3,Sam Hughes,0,50,0,0,134,7,3,2024-08-03,115,97,193,139,240,0,0,5,900,0,1
4,Deiveson Figueiredo,1,50,14,0,150,7,3,2024-08-03,135,45,90,59,104,2,14,2,900,1,1


In [108]:
ufc.describe()

,Knockdown,Significant_Strike_Percent,Takedown_Percent,Submission_Attempt,Ground_Control,win_by,No_of_rounds,Weight Division,Significant_Strikes_Landed,Significant_Strikes_Attempted,Total_Strikes_Landed,Total_Strikes_Attempted,Takedowns_Landed,Takedowns_Attempted,Ground_Strikes_Landed,time_fought,Gender,Winner
count,14468.000000,14468.000000,14468.000000,14468.000000,14468.000000,14468.000000,14468.000000,14468.000000,14468.000000,14468.000000,14468.000000,14468.000000,14468.000000,14468.000000,14468.000000,14468.000000,14468.000000,14468.000000
mean,0.218966,45.291747,26.528062,0.386646,134.303705,8.365773,3.178878,168.215372,37.146461,83.598562,55.152613,104.192010,1.074025,2.856442,4.964542,640.639066,0.894526,0.499793
std,0.496223,16.489580,35.187764,0.834849,174.799960,1.361071,0.570772,38.090092,32.153304,70.329278,44.170592,78.291762,1.698619,3.741720,9.070970,360.696798,0.307174,0.500017
min,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,3.000000,115.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000
25%,0.000000,35.000000,0.000000,0.000000,6.000000,7.000000,3.000000,145.000000,13.000000,29.000000,20.000000,39.000000,0.000000,0.000000,0.000000,287.000000,1.000000,0.000000
50%,0.000000,45.000000,0.000000,0.000000,60.000000,9.000000,3.000000,155.000000,29.000000,67.000000,47.000000,92.000000,0.000000,1.000000,1.000000,849.000000,1.000000,0.000000
75%,0.000000,55.000000,50.000000,0.000000,198.000000,10.000000,3.000000,185.000000,53.000000,120.000000,80.000000,152.000000,2.000000,4.000000,6.000000,900.000000,1.000000,1.000000
max,5.000000,100.000000,100.000000,10.000000,1342.000000,10.000000,5.000000,265.000000,445.000000,744.000000,447.000000,746.000000,21.000000,49.000000,136.000000,1500.000000,1.000000,1.000000


In [109]:
date_column=ufc.pop('date')
ufc.insert(0,'date',date_column)

In [110]:
ufc2=ufc.copy()

#Standardization

In [111]:
ufc2.columns[2:-1]

Index(['Knockdown', 'Significant_Strike_Percent', 'Takedown_Percent',
       'Submission_Attempt', 'Ground_Control', 'win_by', 'No_of_rounds',
       'Weight Division', 'Significant_Strikes_Landed',
       'Significant_Strikes_Attempted', 'Total_Strikes_Landed',
       'Total_Strikes_Attempted', 'Takedowns_Landed', 'Takedowns_Attempted',
       'Ground_Strikes_Landed', 'time_fought', 'Gender'],
      dtype='object')

In [112]:
columns_to_standardize=ufc2.columns[2:-1].tolist()
scaler=StandardScaler()
scaler.fit(ufc2[columns_to_standardize])
ufc2[columns_to_standardize]=scaler.transform(ufc2[columns_to_standardize])

ufc2.head()

,date,fighter,Knockdown,Significant_Strike_Percent,Takedown_Percent,Submission_Attempt,Ground_Control,win_by,No_of_rounds,Weight Division,Significant_Strikes_Landed,Significant_Strikes_Attempted,Total_Strikes_Landed,Total_Strikes_Attempted,Takedowns_Landed,Takedowns_Attempted,Ground_Strikes_Landed,time_fought,Gender,Winner
0,2024-08-03,Cory Sandhagen,-0.441280,-0.078340,-0.753926,-0.463149,-0.699702,-1.003489,3.190742,-0.872051,1.052915,1.072157,0.562552,0.866123,-0.632315,-0.228898,-0.547319,2.382585,0.343381,0
1,2024-08-03,Azamat Murzakanov,1.574012,0.406832,-0.753926,-0.463149,-0.516630,1.200733,-0.313407,0.965760,0.244261,0.019927,-0.161937,-0.206823,-0.632315,-0.496164,-0.326828,-0.395468,0.343381,1
2,2024-08-03,Denis Tiuliulin,-0.441280,-0.502865,0.951271,-0.463149,0.072636,-1.003489,-0.313407,0.440671,-0.346678,-0.193362,2.894501,1.811339,1.133887,0.572900,-0.547319,0.719080,0.343381,0
3,2024-08-03,Sam Hughes,-0.441280,0.285539,-0.753926,-0.463149,-0.001738,-1.003489,-0.313407,-1.397141,1.861569,1.555614,1.898328,1.734700,-0.632315,-0.763430,0.003909,0.719080,-2.912215,1
4,2024-08-03,Deiveson Figueiredo,1.574012,0.285539,-0.356047,-0.463149,0.089799,-1.003489,-0.313407,-0.872051,0.244261,0.091024,0.087106,-0.002453,0.545153,2.978294,-0.326828,0.719080,0.343381,1


#Train Test Split

In [113]:
X=ufc2.iloc[:,2:-1].values
y=ufc2.iloc[:,-1].values

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [114]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators = 50, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=50, random_state=0)

In [115]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1 1]
 [1 0]
 [0 0]
 ...
 [1 1]
 [0 0]
 [0 1]]


In [116]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm2 = confusion_matrix(y_test, y_pred)
print(cm2)
accuracy_score(y_test, y_pred)

[[1198  242]
 [ 230 1224]]


0.8369039391845197

#Prediction System

In [117]:
ufc3=ufc2.copy()

to_drop=['Winner']
ufc3.drop(to_drop,axis=1,inplace=True)

ufc3.head()

,date,fighter,Knockdown,Significant_Strike_Percent,Takedown_Percent,Submission_Attempt,Ground_Control,win_by,No_of_rounds,Weight Division,Significant_Strikes_Landed,Significant_Strikes_Attempted,Total_Strikes_Landed,Total_Strikes_Attempted,Takedowns_Landed,Takedowns_Attempted,Ground_Strikes_Landed,time_fought,Gender
0,2024-08-03,Cory Sandhagen,-0.441280,-0.078340,-0.753926,-0.463149,-0.699702,-1.003489,3.190742,-0.872051,1.052915,1.072157,0.562552,0.866123,-0.632315,-0.228898,-0.547319,2.382585,0.343381
1,2024-08-03,Azamat Murzakanov,1.574012,0.406832,-0.753926,-0.463149,-0.516630,1.200733,-0.313407,0.965760,0.244261,0.019927,-0.161937,-0.206823,-0.632315,-0.496164,-0.326828,-0.395468,0.343381
2,2024-08-03,Denis Tiuliulin,-0.441280,-0.502865,0.951271,-0.463149,0.072636,-1.003489,-0.313407,0.440671,-0.346678,-0.193362,2.894501,1.811339,1.133887,0.572900,-0.547319,0.719080,0.343381
3,2024-08-03,Sam Hughes,-0.441280,0.285539,-0.753926,-0.463149,-0.001738,-1.003489,-0.313407,-1.397141,1.861569,1.555614,1.898328,1.734700,-0.632315,-0.763430,0.003909,0.719080,-2.912215
4,2024-08-03,Deiveson Figueiredo,1.574012,0.285539,-0.356047,-0.463149,0.089799,-1.003489,-0.313407,-0.872051,0.244261,0.091024,0.087106,-0.002453,0.545153,2.978294,-0.326828,0.719080,0.343381


In [118]:
def calculate_avg_attributes(df, years):
  temp_df=df.sort_values(by='date',ascending=False)
  grouped_df=temp_df.groupby('fighter').head(years)

  exclude_columns=['Weight Division','Gender']
  avg_attributes = grouped_df.groupby('fighter').mean(numeric_only=True)
  last_values=grouped_df.groupby('fighter')[exclude_columns].last()

  avg_attributes[exclude_columns]=last_values
  avg_attributes=avg_attributes.reset_index()
  return avg_attributes

avg_attributes_df=calculate_avg_attributes(ufc3, 3)
avg_attributes_df.head()

,fighter,Knockdown,Significant_Strike_Percent,Takedown_Percent,Submission_Attempt,Ground_Control,win_by,No_of_rounds,Weight Division,Significant_Strikes_Landed,Significant_Strikes_Attempted,Total_Strikes_Landed,Total_Strikes_Attempted,Takedowns_Landed,Takedowns_Attempted,Ground_Strikes_Landed,time_fought,Gender
0,Alatengheili,0.230484,-0.381572,0.240772,-0.463149,0.560830,-0.268748,-0.313407,-0.872051,-0.388148,-0.207582,0.170120,0.201918,0.741397,0.751077,-0.400325,-0.069236,0.343381
1,Aoriqileng,0.902248,0.831357,0.667071,-0.463149,0.061194,-0.268748,-0.313407,-0.872051,0.430874,0.053106,0.796501,0.402031,-0.043581,-0.050720,0.922622,0.042588,0.343381
2,Maheshate,1.574012,0.255216,-0.753926,-0.463149,-0.768355,1.200733,-0.313407,-0.346962,-0.548842,-0.342665,-0.795865,-0.564472,-0.632315,-0.496164,-0.547319,-1.040076,0.343381
3,Mizuki,-0.441280,0.406832,0.667071,-0.463149,0.438781,-1.003489,-0.313407,-1.397141,0.555282,0.304314,0.630473,0.457382,-0.043581,-0.228898,-0.437073,0.719080,-2.912215
4,Rongzhu,0.566366,0.315862,0.667071,-0.463149,0.910766,0.833363,-0.313407,-0.346962,0.928507,0.681126,0.822915,0.763938,1.428254,0.840166,1.822962,0.407173,0.343381


In [119]:
def get_fighter_row(df, fighter):
  row=df.loc[df['fighter']==fighter]
  row_values=row.values.flatten()[1:]
  return row_values


def get_win_probability(fighter):
  fighter_row=get_fighter_row(avg_attributes_df, fighter)
  prediction=classifier.predict_proba([fighter_row])
  return prediction[0][1]

In [120]:
get_win_probability('Tony Ferguson')

0.34

In [121]:
def predict_fight(r_fighter, b_fighter):
  r_prob=get_win_probability(r_fighter)
  b_prob=get_win_probability(b_fighter)

  r_win=round(r_prob/(r_prob+b_prob),2)
  b_win=round(b_prob/(r_prob+b_prob),2)

  return r_win, b_win

In [122]:
predict_fight('Shavkat Rakhmonov','Belal Muhammad')

(0.57, 0.43)